In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/OneDrive-JohnsHopkins/000Projects/0000-Infrastructure/0000-RecFld/FieldNN


# Batch

In [2]:
import os
from fieldnn.dataset import RFGDataset, my_collate_fn
from torch.utils.data import DataLoader

Tensor_folder = 'data/ProcData/FldGrnTensor/'
recfldgrn_list = ['P-InfoGrn',  'P-EC-PNSect-TknzGrn']
full_recfldgrn_list = ['B-' + i for i in recfldgrn_list]

# from the get_grain_fn to get the Elig_Set.
Elig_Set = ['P4', 'P5', 'P6', 'P7']

dataset = RFGDataset(Tensor_folder, recfldgrn_list, Elig_Set, RecRootID = 'PID')
print(len(dataset), '<---- dataset')
dataloader = DataLoader(dataset, batch_size = 4, shuffle = True, collate_fn = my_collate_fn)
print(len(dataloader), '<---- dataset')


for idx, batch in enumerate(dataloader):
    # print(f'\n------ {idx}')
    batch_rfg, batch_y = batch
    for k, v in batch_rfg.items(): print(k, v.shape)
    break
    # for k, v in batch_rfg.items(): print(k, v.shape)
    # print(batch_y.shape)

4 <---- dataset
1 <---- dataset
B-P-InfoGrn_wgt torch.Size([4, 43])
B-P-InfoGrn_tknidx torch.Size([4, 43])
B-P-InfoGrn_fldidx torch.Size([4, 43])
B-P-EC-PNSect-TknzGrn_wgt torch.Size([4, 23, 14, 221])
B-P-EC-PNSect-TknzGrn_tknidx torch.Size([4, 23, 14, 221])
B-P-EC-PNSect-TknzGrn_fldidx torch.Size([4, 23, 14, 221])


In [3]:
# Prepare the Input
RECFLD_TO_TENSOR = {}
for full_recfldgrn in full_recfldgrn_list:
    RECFLD_TO_TENSOR[full_recfldgrn] = {k: v for k, v in batch_rfg.items() if full_recfldgrn in k}
    
[i for i in RECFLD_TO_TENSOR]

['B-P-InfoGrn', 'B-P-EC-PNSect-TknzGrn']

In [4]:
from fieldnn.dataflowfn.embedflowfn import get_EmbeddingBlock_SubUnit

from fieldnn.dataflowfn.baseflowfn import mapping_SubUnitName_to_SubUnitNNList
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_Default_NNPara_List
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_BasicNN_Config_List

############################################# Hyperparameters
default_BasicNNtype_To_NNName = {
    'expander': None, # will be updated according to the Grn Type
    'reducer': 'Max',
    'merger': 'Merger',
    'learner': None, # TODO: ignore this currently
    
}
#############################################

############################
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}
############################


default_SubUnitName = 'E'
fldgrn_folder = 'data/ProcData/FldGrnInfo'
learner_default_dict = {} # To update it in the future. 


df_SubUnit = get_EmbeddingBlock_SubUnit(full_recfldgrn_list, default_SubUnitName)

s = df_SubUnit.apply(lambda x: mapping_SubUnitName_to_SubUnitNNList(x['SubUnitName'], 
                                                                    x['input_names'],
                                                                    default_BasicNNtype_To_NNName), 
                    axis = 1)
df_SubUnit['SubUnit_BasicNN_List'] = s
s = df_SubUnit.apply(lambda x: get_SubUnit_Default_NNPara_List(x['SubUnit_BasicNN_List'], 
                                                               x['input_names'],
                                                               fldgrn_folder, 
                                                               learner_default_dict), axis = 1)

df_SubUnit['SubUnit_DefaultBasicNN_List'] = s



s = df_SubUnit.apply(lambda x: get_SubUnit_BasicNN_Config_List(x['SubUnit_BasicNN_List'], 
                                                               x['SubUnit_DefaultBasicNN_List'], 
                                                               x['input_names'], 
                                                               x['output_name'], 
                                                                embed_size, 
                                                                process, 
                                                               ), axis = 1)

df_SubUnit['SubUnit_BasicNN_Config_List'] = s
df_SubUnit

,SubUnitName,input_names,output_name,output_layerid,SubUnit_BasicNN_List,SubUnit_DefaultBasicNN_List,SubUnit_BasicNN_Config_List
0,E,[B-P-InfoGrn],B-P-Info,3,[expander-CateEmbed],"[{'full_recfldgrn': 'B-P-InfoGrn', 'Info': ['P...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
1,E,[B-P-EC-PNSect-TknzGrn],B-P-EC-PNSect-Tknz,5,[expander-LLMEmbed],"[{'full_recfldgrn': 'B-P-EC-PNSect-TknzGrn', '...","[{'nn_type_nn_name': 'expander-LLMEmbed', 'Bas..."


In [ ]:
from fieldnn.module.embedblock import EmbedBlockLayer

EmbedBlock = EmbedBlockLayer(df_SubUnit)

RECFLD_TO_EMBEDTESNOR = EmbedBlock(RECFLD_TO_TENSOR)

for k, v in RECFLD_TO_EMBEDTESNOR.items():
    print(k, v['info'].shape)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Get Input Tensors

In [1]:
for full_recfld, info_dict in RECFLD_TO_EMBEDTESNOR.items():
    print(full_recfld, info_dict['info'].shape)

In [ ]:
max_layer = max([len(i.split('-')) for i in RECFLD_TO_EMBEDTESNOR])
max_layer

# Get df DataFlow

In [ ]:
# # fieldnn.utils.datanamefn.py

# def get_curfld_recinfo(curfld):
#     '''
#         a helper function that get the rec information from the current recfldgrn name.
#         the input is: prefix_rec@fld_grn
#         we want to check whether @ is here. 
#         If there is a @ in the current level, we want to put the rec as the potential merger target. 
#     '''
#     if '@' in curfld:
#         fld_list = curfld.split('@')
#         return '@'.join(fld_list[:-1]), fld_list[-1]
#     else:
#         return None, curfld

In [2]:
# fieldnn.dataflowfn.reprflowfn.py

import numpy as np
import pandas as pd


def get_Repr_dataflow_table(full_recfldgrn_list):
    '''
        input: a list of full_recfldgrn
        output: the dataframe that shows the data flow from the grain-embedding-tensor to final feature vector.
    '''
    max_layer = max([len(i.split('-')) for i in full_recfldgrn_list])
    L = []
    for full_recfldgrn in full_recfldgrn_list:
        d = {}
        recfldgrn = '-'.join(full_recfldgrn.split('-')[-2:])
        current_layer_id = len(full_recfldgrn.split('-'))
        d['recfldgrn'] = recfldgrn
        for layer_id in range(max_layer, 1, -1):
            if layer_id > current_layer_id:
                d[layer_id] = np.nan
            elif layer_id == current_layer_id:
                d[layer_id] = full_recfldgrn
            else:
                d[layer_id] = np.nan
        L.append(d)
    df_dataflow = pd.DataFrame(L).set_index('recfldgrn')
    return df_dataflow


In [44]:
def update_df_Repr_dataflow(df):
    
    df_dataflow = df.copy()

    for layer_idx in df_dataflow.columns:
        
        # from layerid: full_recfldgrn to layerid-1: pfx_rec if pfx_rec is unique.
        for index in df_dataflow.index:
            full_recfldgrn = df_dataflow.loc[index, layer_idx] 
            if pd.isna(full_recfldgrn): continue
            # if full_recfldgrn == 'ToFill': continue
            pfx_rec = '-'.join(full_recfldgrn.split('-')[:-1])
            cur_rec = full_recfldgrn.split('-')[-1]
            s = df_dataflow.loc[df_dataflow.index != index, layer_idx]
            current_fullrfg_list = s[-s.isna()].to_list()
            current_pfxrec_list = ['-'.join(i.split('-')[:-1]) for i in current_fullrfg_list]

            # print(current_pfxrec_list)
            if pfx_rec not in current_pfxrec_list:
                pfx_rec_list = pfx_rec.split('-')
                pfx_rec_list[-1] = pfx_rec_list[-1].replace('@', '')
                output_recfldgrn = '-'.join(pfx_rec_list)
                df_dataflow.loc[index, layer_idx - 1] = output_recfldgrn
        
        # create new merge index.
        s = df_dataflow.loc[:, layer_idx]
        current_fullrfg_list = s[-s.isna()].to_list()
        if len(current_fullrfg_list) == 0: continue
        current_pfxrec_list = [{'i': i, 'j': '-'.join(i.split('-')[:-1])} for i in current_fullrfg_list]
        dfx = pd.DataFrame(current_pfxrec_list)
        s = dfx.groupby('j').apply(lambda x: x['i'].to_list()).to_dict()
        s = {k: v for k, v in s.items() if len(v) > 1}
        # print(s)
        for pfx_rec, fullrec_list in s.items():
            
            full_recfldgrn_new = pfx_rec + '-' + '&'.join([i.split('-')[-1] for i in fullrec_list])
            new_index = '(Merge)' + full_recfldgrn_new
            l = list(df_dataflow.index)
            
            # print(fullrec_list, '<----')
            # print(df_dataflow[layer_idx].to_list(), '<----')
            
            selected_index_list = df_dataflow[df_dataflow[layer_idx].isin(fullrec_list)].index
            # print(selected_index_list)
            loc = max([l.index(i) for i in selected_index_list])
            l.insert(loc + 1, new_index)
            
            df_dataflow.loc[new_index, layer_idx] = full_recfldgrn_new
            df_dataflow.loc[new_index, layer_idx-1] = pfx_rec
            df_dataflow = df_dataflow.reindex(l)
            
            
    df_dataflow = df_dataflow.iloc[:, :-1]
            
    return df_dataflow

In [45]:
def update_df_Repr_dataflow_completename(df):
    df_dataflow = df.copy()
    L = []
    for recfldgrn, row in df_dataflow.iterrows():
        # print(recfldgrn)
        new_row = {}
        new_row['recfldgrn'] = recfldgrn
        full_recfldgrn = [i for i in row.values if not pd.isna(i)][0]
        curlayer_idx = len(full_recfldgrn.split('-'))
        for layer_idx, full_recfldgrn in row.items():
            if curlayer_idx < layer_idx:
                new_row[layer_idx] = np.nan
            elif curlayer_idx == layer_idx:
                new_row[layer_idx] = full_recfldgrn
            elif curlayer_idx > layer_idx:
                last = new_row[layer_idx+1]
                output_fullname = '-'.join(last.split('-')[:-1]) + '@' + last.split('-')[- 1]
                new_row[layer_idx] = output_fullname
            else:
                raise ValueError('wrong information')
        L.append(new_row)
    df_dataflow_filled = pd.DataFrame(L).set_index('recfldgrn')
    return df_dataflow_filled

## Test

In [53]:
# full_recfldgrn_list = [i for i in RECFLD_TO_EMBEDTESNOR]
full_recfldgrn_list = ['B-P-Info', 
                       'B-P-EC-PNSect-Tknz']

df_dataflow = get_Repr_dataflow_table(full_recfldgrn_list)
df_dataflow_new = update_df_Repr_dataflow(df_dataflow)
df_dataflow_new

,5,4,3,2
recfldgrn,,,,
P-Info,NaN,NaN,B-P-Info,NaN
PNSect-Tknz,B-P-EC-PNSect-Tknz,B-P-EC-PNSect,B-P-EC,NaN
(Merge)B-P-Info&EC,NaN,NaN,B-P-Info&EC,B-P


In [87]:
# full_recfldgrn_list = [i for i in RECFLD_TO_EMBEDTESNOR]
full_recfldgrn_list = ['B-P-Info', 
                       'B-P-EC-EHR', 
                       'B-P-EC-PNSect-Tknz']

df_dataflow = get_Repr_dataflow_table(full_recfldgrn_list)
df_dataflow_new = update_df_Repr_dataflow(df_dataflow)
df_dataflow_new

,5,4,3,2
recfldgrn,,,,
P-Info,NaN,NaN,B-P-Info,NaN
EC-EHR,NaN,B-P-EC-EHR,NaN,NaN
PNSect-Tknz,B-P-EC-PNSect-Tknz,B-P-EC-PNSect,NaN,NaN
(Merge)B-P-EC-EHR&PNSect,NaN,B-P-EC-EHR&PNSect,B-P-EC,NaN
(Merge)B-P-Info&EC,NaN,NaN,B-P-Info&EC,B-P


In [88]:
# df_dataflow_new = update_df_Repr_dataflow(df_dataflow, style = 'Reducer&Merger')
# df_dataflow_new

# Get df SubUnit

In [89]:
df_dataflow = df_dataflow_new.copy()

In [90]:
layeridx_list = list(df_dataflow.columns)
layeridx_list

[5, 4, 3, 2]

In [91]:
for idx in range(len(layeridx_list) - 1):
    A_layerid = layeridx_list[idx]
    B_layerid = layeridx_list[idx + 1]
    print(A_layerid, B_layerid)

5 4
4 3
3 2


In [92]:
for idx in range(len(layeridx_list) - 1):
    A_layerid = layeridx_list[idx]
    B_layerid = layeridx_list[idx + 1]
    # print(A_layerid, B_layerid)
    
    
    A_tensors = df_dataflow[A_layerid]
    A_tensors = A_tensors[-A_tensors.isna()].to_dict()
    
    B_tensors = df_dataflow[B_layerid]
    B_tensors = B_tensors[-B_tensors.isna()].to_dict()
    
    
    print(f'\nFrom Layer {A_layerid} to {B_layerid}:')
    
    # from A tensor to B tensor, we have the Reducer NNs.
    # also notice that some 
    print('A', A_layerid, A_tensors)

    
    # from B tensor: potential there are some Merger NNs. 
    print('B', B_layerid, B_tensors)


From Layer 5 to 4:
A 5 {'PNSect-Tknz': 'B-P-EC-PNSect-Tknz'}
B 4 {'EC-EHR': 'B-P-EC-EHR', 'PNSect-Tknz': 'B-P-EC-PNSect', '(Merge)B-P-EC-EHR&PNSect': 'B-P-EC-EHR&PNSect'}

From Layer 4 to 3:
A 4 {'EC-EHR': 'B-P-EC-EHR', 'PNSect-Tknz': 'B-P-EC-PNSect', '(Merge)B-P-EC-EHR&PNSect': 'B-P-EC-EHR&PNSect'}
B 3 {'P-Info': 'B-P-Info', '(Merge)B-P-EC-EHR&PNSect': 'B-P-EC', '(Merge)B-P-Info&EC': 'B-P-Info&EC'}

From Layer 3 to 2:
A 3 {'P-Info': 'B-P-Info', '(Merge)B-P-EC-EHR&PNSect': 'B-P-EC', '(Merge)B-P-Info&EC': 'B-P-Info&EC'}
B 2 {'(Merge)B-P-Info&EC': 'B-P'}


In [101]:

def get_Repr_SubUnit_List(df_dataflow, 
                          default_R_subunit_name = 'RL', 
                          default_MR_subunit_name = 'ML', # or 'MLRL'
                         ):
    layeridx_list = list(df_dataflow.columns)
    
    SubUnit_List = []
    for idx in range(len(layeridx_list) - 1):
        A_layerid = layeridx_list[idx]
        B_layerid = layeridx_list[idx + 1]
        # print(A_layerid, B_layerid)

        A_tensors = df_dataflow[A_layerid]
        A_tensors = A_tensors[-A_tensors.isna()].to_dict()

        B_tensors = df_dataflow[B_layerid]
        B_tensors = B_tensors[-B_tensors.isna()].to_dict()
        
        
        # Deal with the Merge First.
        # from B tensor: potential there are some Merger NNs. 
        # print('B', B_tensors)
        # check whether these is a '(Merger') in the key
        # merger_tensors = [v for k, v in B_tensors.items() if '(Merge)' in k and '@' not in v]
        # merger_tensors = [v for k, v in B_tensors.items() if '(Merge)' in k]
        # print('B-merger_tensors', merger_tensors)
        
        merger_tensors = [tensor for index, tensor in A_tensors.items() 
                          if '(Merge)' in index and '&' in tensor]
        
        # print(merger_tensors, '<----- merger_tensors', A_layerid)

        for output_tensor in merger_tensors:
            # print(B_tensors, '<---- B_tensor')
            input_tensors = [i for k, i in A_tensors.items() 
                             if '-'.join(output_tensor.split('-')[:-1]) == '-'.join(i.split('-')[:-1])]
            input_tensors = [i for i in input_tensors if i != output_tensor]
            # print(output_tensor, ':', input_tensors)

            d = {}
            d['SubUnitName'] = default_MR_subunit_name
            d['input_names'] = input_tensors
            d['output_name'] = output_tensor
            
            d['input_layerid'] = A_layerid
            d['output_layerid'] = A_layerid
        
            SubUnit_List.append(d)
        

        # print(f'\nFrom Layer {A_layerid} to {B_layerid}:')

        # from A tensor to B tensor, we have the Reducer NNs.
        # also notice that some 

        # print('A', A_tensors)
        for k in A_tensors:
            # pass the merged tensors. 
            if k not in B_tensors: continue

            input_name = A_tensors[k]
            output_name = B_tensors[k]

            if pd.isna(output_name) == True: continue # pass it. 

            d = {}
            d['SubUnitName'] = default_R_subunit_name
            d['input_names'] = [input_name]
            d['output_name'] = output_name
            
            d['input_layerid'] = A_layerid
            d['output_layerid'] = B_layerid
        
            SubUnit_List.append(d)
    df_SubUnit = pd.DataFrame(SubUnit_List)
    return df_SubUnit




In [102]:
default_R_subunit_name = 'R'
default_MR_subunit_name = 'M'

df_SubUnit = get_Repr_SubUnit_List(df_dataflow, default_R_subunit_name, default_MR_subunit_name)
df_SubUnit

,SubUnitName,input_names,output_name,input_layerid,output_layerid
0,R,[B-P-EC-PNSect-Tknz],B-P-EC-PNSect,5,4
1,M,"[B-P-EC-EHR, B-P-EC-PNSect]",B-P-EC-EHR&PNSect,4,4
2,R,[B-P-EC-EHR&PNSect],B-P-EC,4,3
3,M,"[B-P-Info, B-P-EC]",B-P-Info&EC,3,3
4,R,[B-P-Info&EC],B-P,3,2


In [103]:
df_dataflow

,5,4,3,2
recfldgrn,,,,
P-Info,NaN,NaN,B-P-Info,NaN
EC-EHR,NaN,B-P-EC-EHR,NaN,NaN
PNSect-Tknz,B-P-EC-PNSect-Tknz,B-P-EC-PNSect,NaN,NaN
(Merge)B-P-EC-EHR&PNSect,NaN,B-P-EC-EHR&PNSect,B-P-EC,NaN
(Merge)B-P-Info&EC,NaN,NaN,B-P-Info&EC,B-P
